# Recordati - List
## Beachten
* PDF zuerst entschlüsseln!
* Früher OCR

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import re

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *
import consts

## Unlock PDF

In [2]:
import pikepdf
pdf = pikepdf.open('Disclosure-Form_2019.pdf')
pdf.save('unlocked.pdf')

## Import PDF

In [3]:
# Read pdf into DataFrame
df = tabula.read_pdf("unlocked.pdf", pages='all', lattice=True)

## Format Table

In [28]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['empty0', 'name', 'location', 'country', 'address', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'total']

#Set Type
index_hco = df_export[df_export['name'].str.contains("Number of Recipients", na=False)].index[0]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')

#Copy related expenses to total , if total empty
df_export.loc[df_export.total.isna(), 'tmp'] = df_export['related_expenses']
df_export.loc[df_export.total.isna(), 'total'] = df_export['tmp']
df_export.loc[df_export.tmp.notna(), 'related_expenses'] = np.nan


#Remove rows which have no values
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.dropna(subset=['name'], how='all')
df_export = df_export.reset_index(drop=True)

#Remove empty
df_export.drop(columns=['empty0', 'tmp'], inplace=True)

#Extract PLZ
df_export['plz'] = df_export.address.str.extract(r' (\d{4})')
df_export['address'] = df_export.address.str.replace(r'(,\s\d{4}.*)', '')

#basic string conversion
df_export = basic_string_conversion(df_export)

#Add Fields
df_export = add_uci(df_export)

#Convert to Numbers
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)

#Remove Carination
df_export = remove_carination(df_export, ' ')

#Export
export_list(df_export, 'recordani')


Duplicates found. Please check for duplicates: df_export[df_export.duplicated()]
Total nicht Summe der Werte
saved


In [27]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx', open=True)